------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
- # ****Classification (Task A)****
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

- ## ****The (evaluate_model) function serves as a comprehensive evaluation framework that automates the diagnostic process for classification models. It computes a wide range of statistical metrics, including ROC-AUC, MCC, and Average Precision, which are essential for assessing performance on imbalanced datasets. Furthermore, it generates a four-panel visualization dashboard—consisting of a Normalized Confusion Matrix, ROC Curve, Precision-Recall Curve, and Calibration Curve—to provide deep insights into the model’s predictive reliability, threshold trade-offs, and probability accuracy****

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, precision_recall_curve, average_precision_score,
    confusion_matrix, matthews_corrcoef, ConfusionMatrixDisplay
)
from sklearn.calibration import calibration_curve

def evaluate_model(model, X_test, y_test, model_name="Model"):

    y_pred = model.predict(X_test)
    y_probs = model.predict_proba(X_test)[:, 1]

    metrics = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-score": f1_score(y_test, y_pred),
        "MCC": matthews_corrcoef(y_test, y_pred),
        "ROC-AUC": roc_auc_score(y_test, y_probs),
        "Avg Precision (AP)": average_precision_score(y_test, y_probs)
    }

    print(f"\n--- {model_name} Performance Metrics ---")
    for name, value in metrics.items():
        print(f"{name:22}: {value:.4f}")

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'Evaluation Report: {model_name}', fontsize=16)

    ConfusionMatrixDisplay.from_predictions(y_test, y_pred, ax=axes[0, 0], cmap='Blues', normalize='true')
    axes[0, 0].set_title("Normalized Confusion Matrix")

    # B. ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_probs)
    axes[0, 1].plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {metrics["ROC-AUC"]:.4f}')
    axes[0, 1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    axes[0, 1].set_xlabel('False Positive Rate')
    axes[0, 1].set_ylabel('True Positive Rate')
    axes[0, 1].set_title("ROC Curve")
    axes[0, 1].legend(loc="lower right")

    # C. Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_test, y_probs)
    axes[1, 0].plot(recall, precision, color='purple', lw=2, label=f'AP = {metrics["Avg Precision (AP)"]:.4f}')
    axes[1, 0].set_xlabel('Recall')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].set_title("Precision-Recall Curve")
    axes[1, 0].legend(loc="lower left")

    # D. Calibration Curve
    prob_true, prob_pred = calibration_curve(y_test, y_probs, n_bins=10)
    axes[1, 1].plot(prob_pred, prob_true, marker='o', linewidth=1, label=model_name)
    axes[1, 1].plot([0, 1], [0, 1], linestyle='--', color='gray', label="Perfectly Calibrated")
    axes[1, 1].set_xlabel('Mean Predicted Probability')
    axes[1, 1].set_ylabel('Fraction of Positives')
    axes[1, 1].set_title("Calibration Curve")
    axes[1, 1].legend(loc="lower right")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

    return metrics

- ## ****This code snippet performs the final feature engineering and data preparation steps before model training. It systematically separates the target variable from the predictors while excluding non-informative identifiers like (user_id) and (order_id). To ensure optimal convergence of the Logistic Regression model, a (StandardScaler) is applied to normalize the 76 features. Additionally, the code incorporates memory management by converting data to (float32) and using (gc.collect()) to clear unused variables, which is critical for handling large-scale datasets like Instacart****

In [ ]:
excluded_cols = ['user_id', 'product_id', 'target', 'order_id']
features = [col for col in train_df.columns if col not in excluded_cols]

X_train = train_df[features].astype('float32')
y_train = train_df['target']

X_val = val_df[features].astype('float32')
y_val = val_df['target']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

del X_train, X_val
gc.collect()

print(len(features))

76
